# Agent Engine に ADK のエージェントをデプロイして Trace Explorer でトレースを確認

## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user \
    google-adk==1.2.1 \
    google-cloud-aiplatform==1.96.0

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [2]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Search Agent App の作成

In [2]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from vertexai import agent_engines

from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(
    project=PROJECT_ID, location=LOCATION,
    staging_bucket=f'gs://{PROJECT_ID}'
)
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [3]:
instruction = '''
You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.

[Format instruction]
Output in Japanese, in plain text only.
Avoid adding citation marks such as [1][2].
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [10]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [11]:
client = LocalApp(search_agent)
query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

高田馬場のおすすめカレー店について調べてみます。


 高田馬場には美味しいカレー屋がたくさんありますね！ おすすめのお店をいくつかご紹介します。

*   **カレーハウス 横浜ボンベイ:** サラッとしたルーが特徴で、特に「カシミールカレー」が人気です。野菜の甘みと20種類のスパイスが絶妙にマッチした極辛口カレーです。
*   **カリーライス専門店 エチオピア:** 長時間煮込んだ野菜が美味しいトロトロカレーが楽しめます。平日限定の「ポーク野菜カリー」もおすすめです。
*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。
*   **プネウマカレー:** コスパが良いと評判のお店です。
*   **カレーハウス リオ 高田馬場店:** 2025年6月12日にオープンしたばかりのお店で、三元豚カツカレーや半スパカレー（ナポリタン）がおすすめです。

他にも、Rettyの記事では以下のカレー店がおすすめとして紹介されています。

*   **アプサラ:** スリランカカレーが人気です。
*   **まっちゃんカレー:** ロックなキーマカレーが楽しめます。

食べログでは、以下のカレー店がランキング上位に入っています。

1.  カレーライス専門店 ブラザー
2.  エチオピアカリーキッチン 高田馬場店
3.  プネウマカレー
4.  横浜ボンベイ 高田馬場店
5.  白カレーの店 1/f ゆらぎ

これらの情報を参考に、ぜひ高田馬場でカレー屋さん巡りを楽しんでみてください。



## AgentEngine へのデプロイ

作成したエージェント `search_agent` にトレーシング機能を付与して、Agent Engine にデプロイします。


In [6]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(
    agent=search_agent,
    enable_tracing=True,
)  

remote_agent = agent_engines.create(
    agent_engine=app,
    display_name='interactive_agent',
    requirements=[
        'google-adk==1.2.1'
    ]
)

Identified the following requirements: {'pydantic': '2.11.5', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.96.0'}
The following requirements are missing: {'pydantic', 'cloudpickle', 'google-cloud-aiplatform'}
The following requirements are appended: {'pydantic==2.11.5', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-adk==1.2.1', 'pydantic==2.11.5', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/2504115742027284480/operations/4417645134070415360
View progress and logs at https://console.cloud.google.com/logs/query?project=etsuji-15pro-poc
AgentEngine created. Resource name: projects/879055303739/locations/us-centra

デプロイしたエージェントの ID を確認します。この ID は後ほど使用します。

In [7]:
print(remote_agent.name)

2504115742027284480


## 実行例

### ノートブック上で試す場合

In [8]:
class RemoteApp:
    def __init__(self, remote_agent, user_id='default_user'):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)
    
    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [12]:
remote_client = RemoteApp(remote_agent)
query = '''
今年のゴールデンウィークは、何連休でしょうか？
'''
_ = remote_client.stream(query)

 今年のゴールデンウィーク（2025年）は、祝日の並びがあまり良くありません。

*   **カレンダー通りの場合:** 5月3日（憲法記念日）～6日（みどりの日の振替休日）の4連休がメインとなります。
*   **最大:** 4月28日(月)と、4月30日(水)～5月2日（金）に休みを取ると、最大11連休にすることも可能です。
*   **8連休:** 4月30日(水)、5月1日(木)・2日(金)に休みを取れば、4月29日(火・祝)から5月7日(火・振休)まで8連休にできます。
*   **前半・後半に分かれた4連休:** 4月28日(月)だけでも休めれば、4月26日(土)～29日(火・昭和の日)と、5月3日(土・憲法記念日)～6日(火・振替休日)にそれぞれ4連休を取れます。


In [13]:
query = '''
来年はどうなりますか？
'''
_ = remote_client.stream(query)

 来年（2026年）のゴールデンウィークは、5月2日（土）から5月6日（水）までの5連休です。



### ADK web の UI を使用する場合

GUI のチャット画面（ADK web）を試したい場合は、Cloud Shell から次の手順で試す事ができます。

※ あくまでお試し用の手順なので、ADK web のすべての機能は使用できません。簡易的な動作確認として利用してください。

1. 作業用ディレクトリ `workdir` を作成して、カレントディレクトリに変更します。

```
mkdir workdir
cd workdir
```

2. `google-adk` のパッケージをインストールします。

```
python -m venv .venv
source .venv/bin/activate
pip install google-adk==1.2.1
```

3. リモートエージェントに接続するコードを用意します。

```
mkdir agent
cat <<EOF >agent/agent.py
import os
from uuid import uuid4
from dotenv import load_dotenv
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.llm_agent import LlmAgent
from google.genai.types import Content, Part

import vertexai
from vertexai import agent_engines

load_dotenv('.env')
PROJECT_ID = os.environ['PROJECT_ID']
AGENT_ID = os.environ['AGENT_ID']
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)
remote_agent = agent_engines.get(AGENT_ID)

async def call_remote_agent(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> LlmResponse:
    session = remote_agent.create_session(user_id='default_user')
    events = remote_agent.stream_query(
                user_id='default_user',
                session_id=session['id'],
                message=str(llm_request.contents)
             )
    content = list(events)[-1]['content']
    remote_agent.delete_session(
        user_id='default_user',
        session_id=session['id'],
    )
    return LlmResponse(content=content)

root_agent = LlmAgent(
    name='remote_agent_proxy',
    model='gemini-2.0-flash', # not used
    description='Interactive agent',
    before_model_callback=call_remote_agent,
)
EOF
```

4. 設定ファイル `agent/.env` を次の内容で作成します。（`your project ID` と `your agent ID` は実際のプロジェクト ID と先ほど確認したエージェントの ID を記入します。）

```
PROJECT_ID="your project ID"
AGENT_ID="your agent ID"
```

5. チャットアプリ（ADK web）を起動します。

```
adk web
```

6. Cloud Shell の「Web でプレビュー」ボタンからポート 8000 に接続して使用します。

## トレースの確認

Cloud Console の [Trace Explorer](https://console.cloud.google.com/traces/list) から Agent Engine 上で実行されたエージェントのトレースが確認できます。

## 後片付け

デプロイしたエージェントを削除します。

In [14]:
remote_agent.delete(force=True)

Delete Agent Engine backing LRO: projects/879055303739/locations/us-central1/operations/2578769111219699712
Agent Engine deleted. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/2504115742027284480


デプロイ済みのエージェントをまとめて削除する場合は、次を実行します。

In [ ]:
from vertexai import agent_engines
for agent in agent_engines.list():
    print(f'{agent.display_name}, {agent.gca_resource.name}')
    agent.delete(force=True)